In [7]:
import pandas as pd
hosts = pd.read_csv("../common/hosts.csv")
supressed_columns = ['ip','user']

In [8]:
consul_hosts = hosts[(hosts.hostname == "nuc-03")]
consul_hosts['ip'].to_csv(r'consul-ip.txt', header=False, index=None, sep=' ')
consul_hosts.drop(columns=supressed_columns)

,hostname,arch,gpu
6,nuc-03,x86_64,0


In [9]:
%%bash

mkdir ~/repos
cd ~/repos
rm -rf ~/repos/home-mesh-tools
git clone https://github.com/6za/home-mesh-tools.git

mkdir: cannot create directory '/root/repos': File exists
Cloning into 'home-mesh-tools'...


In [10]:
%%bash
export HOME_DIR=$PWD
echo $HOME_DIR
source ../common/env.sh 
while read ip; do
    cd ~/repos/home-mesh-tools
    export DOCKER_HOST="tcp://$ip:2376"
    docker-compose -f docker-compose-consul.yaml down 2>/dev/null
    docker-compose -f docker-compose-consul.yaml up -d 2>/dev/null
done <consul-ip.txt
cd $HOME_DIR

/root/08-deploy-basic-infra


In [11]:
%%bash
echo "ip|id|image|ports|runnning" > containers.csv
source ../common/env.sh 
while read ip; do
    export DOCKER_HOST="tcp://$ip:2376"
    docker ps --filter  --format "$ip|{{.ID}}|{{.Image}}|{{.Ports}}|{{.RunningFor}}"  >> containers.csv 2>/dev/null || :
done <consul-ip.txt

In [12]:
containers = pd.read_csv("containers.csv", sep = '|')
pd.merge(containers, hosts, on='ip', how='inner').drop(columns=supressed_columns)

,id,image,ports,runnning,hostname,arch,gpu


In [13]:
import os
import subprocess

for index, row in hosts.iterrows():
    result = subprocess.check_output('ssh -n -o StrictHostKeyChecking=no ' +row['user'] +'@'+row['ip']  +' ps -ef | grep dockerd | grep -v raspi-manager  || :', shell=True, encoding='utf-8')
    if( len(result) > 1 ):
        print(row['hostname'] + "-- Using raspi-manager consul" )
    else :
        print(row['hostname'] + "-- Not using raspi-manager" )

pi-node1-- Not using raspi-manager
pi-node2-- Not using raspi-manager
pi-node3-- Not using raspi-manager
kx-Lenovo-H520g-- Not using raspi-manager
nuc-01-- Not using raspi-manager
nuc-02-- Not using raspi-manager
nuc-03-- Using raspi-manager consul
pi-node4-- Not using raspi-manager
pi-node5-- Not using raspi-manager
pi-node6-- Not using raspi-manager
pi-node7-- Not using raspi-manager
pi-node8-- Not using raspi-manager
tegra-ubuntu-- Using raspi-manager consul


`nuc-03` - is the consul, so it can't use as it would point to itself.
`tegra-ubuntu` - is Jetson, its docker seems to not support consul parameter for `dockerd`

In [14]:
for index, row in hosts.iterrows():
    result = subprocess.check_output('ssh -n -o StrictHostKeyChecking=no ' +row['user'] +'@'+row['ip']  +' cat /etc/hosts | grep raspi-manager  || :', shell=True, encoding='utf-8')
    if( len(result) > 1 ):
        print(row['hostname'] + "-- OK" )
    else :
        print(row['hostname'] + "-- NOK" )
    
# sudo echo "IP  raspi-manager ">> /etc/hosts    


pi-node1-- NOK
pi-node2-- OK
pi-node3-- OK
kx-Lenovo-H520g-- OK
nuc-01-- OK
nuc-02-- OK
nuc-03-- OK
pi-node4-- OK
pi-node5-- OK
pi-node6-- NOK
pi-node7-- OK
pi-node8-- OK
tegra-ubuntu-- OK


### Notes

On most nodes, the dockerd setup is something like this:


```text

ExecStart=/usr/bin/dockerd  --cluster-store=consul://raspi-manager:8500  --cluster-advertise=eno1:2376   -H=0.0.0.0:2376 --tlsverify=true --tlscacert=/certs/ca.pem --tlscert=/certs/cert.pem --tlskey=/certs/key.pem  --experimental=true --metrics-addr="0.0.0.0:9323" -H fd:// --containerd=/run/containerd/containerd.sock

```

Process used to create certs to be used by docker host: https://github.com/paulczar/omgwtfssl